# Custom Layers

deep learning covers a wide range of layers for handling images, and text
that can be composed in creative ways
to design architectures suitable
for a wide variety of tasks.

This Jupyter shows how to build a custom layer that does not exist yet in the deep learning framework.

## (**Layers without Parameters**)

To start, we construct a custom layer
that does not have any parameters of its own.

The following `CenteredLayer` class simply
subtracts the mean from its input.

To build it, we simply need to inherit
from the base layer class and implement the forward propagation function.


In [ ]:
import tensorflow as tf

class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

Let us verify that our layer works as intended by feeding some data through it.


In [ ]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

We can now [**incorporate our layer as a component
in constructing more complex models.**]


In [ ]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

Let's send random data through the network and check that the mean is in fact 0.



In [ ]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

## [**Layers with Parameters**]



We can use built-in functions to create parameters, which
provide some basic functionalities to  access, initialize,
share, save, and load model parameters.

This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let us implement our own version of the fully-connected layer with parameters that can be adjusted through training.

Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.

In this implementation, we bake in the ReLU activation as a default.
This layer requires to input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.


In [ ]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(
            name='weight', shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(name='bias', shape=[self.units],
                                    initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

Next, we instantiate the `MyDense` class
and access its model parameters.


In [ ]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

We can [**directly carry out forward propagation calculations using custom layers.**]


In [ ]:
dense(tf.random.uniform((2, 5)))

We can also (**construct models using custom layers.**)

Once we have that we can use it just like the built-in fully-connected layer.

In [ ]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

## Summary

* We can design custom layers via the basic layer class. This allows us to define flexible new layers that behave differently from any existing layers in the library.
* Once defined, custom layers can be invoked in arbitrary contexts and architectures.
* Layers can have local parameters, which can be created through built-in functions.


## Exercises (Optional)

1. Design a layer that takes an input and computes a tensor reduction,
   i.e., it returns $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
1. Design a layer that returns the leading half of the Fourier coefficients of the data.
